In [42]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from pandas.tools.plotting import scatter_matrix
import matplotlib.pylab as pl
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [ ]:
data = pd.read_excel("https://www.foreignlaborcert.doleta.gov/docs/py2015q4/H-1B_Disclosure_Data_FY15_Q4.xlsx")

In [2]:
def getcsv():
    '''The function downloads the 'H1B data'
    Author: vys217
    '''
    print ("Downloading")
    
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + "H-1B_Disclosure_Data_FY15_Q4.xlsx"):
        if os.path.isfile("H-1B_Disclosure_Data_FY15_Q4.xlsx"):
            # if in the current dir just move it
            if os.system("mv " + "H-1B_Disclosure_Data_FY15_Q4.xlsx " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + "H-1B_Disclosure_Data_FY15_Q4.xlsx"):
                if not os.path.isfile("H-1B_Disclosure_Data_FY15_Q4.xlsx"):
                    os.system("wget https://www.foreignlaborcert.doleta.gov/docs/py2015q4/H-1B_Disclosure_Data_FY15_Q4.xlsx")
                    
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + "H-1B_Disclosure_Data_FY15_Q4.xlsx " + os.getenv("PUIDATA"))

    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + "H-1B_Disclosure_Data_FY15_Q4.xlsx"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")
        
    print ("Downloading")
    
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + "2017_NAICS_Structure.xlsx"):
        if os.path.isfile("2017_NAICS_Structure.xlsx"):
            # if in the current dir just move it
            if os.system("mv " + "2017_NAICS_Structure.xlsx " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + "2017_NAICS_Structure.xlsx"):
                if not os.path.isfile("2017_NAICS_Structure.xlsx"):
                    os.system("wget http://www.census.gov/eos/www/naics/2017NAICS/2017_NAICS_Structure.xlsx")
                    
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + "2017_NAICS_Structure.xlsx " + os.getenv("PUIDATA"))

    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + "2017_NAICS_Structure.xlsx"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")    

In [3]:
getcsv()

Downloading
file in place, you can continue
Downloading
file in place, you can continue


In [4]:
data = pd.read_excel(os.getenv('PUIDATA')+'/'+"H-1B_Disclosure_Data_FY15_Q4.xlsx")

In [5]:
codes = pd.read_excel( os.getenv('PUIDATA')+'/'+"2017_NAICS_Structure.xlsx", header= 2)
codes.drop(['Change Indicator'], axis= 1, inplace= True)
codes.rename(columns={'2017 NAICS Code' : 'NAIC_CODE'}, inplace=True)

In [6]:
data['decision_date'] = data.DECISION_DATE - data.CASE_SUBMITTED
data = data[data.VISA_CLASS == 'H-1B']

In [7]:
data1 = data.copy()

In [8]:
data1.drop(['EMPLOYER_ADDRESS1', 'EMPLOYER_ADDRESS2', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 
                     'EMPLOYER_POSTAL_CODE', 'VISA_CLASS', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE',
                     'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY',
                     'AGENT_ATTORNEY_STATE', 'SOC_CODE', 'PW_WAGE_SOURCE', 'PW_WAGE_SOURCE_YEAR', 
                     'PW_WAGE_SOURCE_OTHER', 'WORKSITE_COUNTY'], axis = 1, inplace = True)

In [10]:
data1 = data1[(data.CASE_STATUS == 'CERTIFIED') | (data1.CASE_STATUS == 'DENIED')]

In [14]:
data1.dtypes

CASE_NUMBER                       object
CASE_STATUS                       object
CASE_SUBMITTED            datetime64[ns]
DECISION_DATE             datetime64[ns]
EMPLOYMENT_START_DATE             object
EMPLOYMENT_END_DATE               object
EMPLOYER_NAME                     object
JOB_TITLE                         object
SOC_NAME                          object
NAIC_CODE                        float64
TOTAL WORKERS                    float64
FULL_TIME_POSITION                object
PREVAILING_WAGE                  float64
PW_UNIT_OF_PAY                    object
PW_WAGE_LEVEL                     object
WAGE_RATE_OF_PAY                  object
WAGE_UNIT_OF_PAY                  object
H-1B_DEPENDENT                    object
WILLFUL VIOLATOR                  object
WORKSITE_CITY                     object
WORKSITE_STATE                    object
WORKSITE_POSTAL_CODE              object
decision_date            timedelta64[ns]
dtype: object

In [15]:
data2 = data1.dropna(subset=['WAGE_UNIT_OF_PAY'])

In [16]:
def convertwage(wage, unit, FT):
   if unit == 'Year':
       return wage
   elif unit == 'Month':
       return wage * 12
   elif unit == 'Week':
       return wage * 4 *12
   elif unit == 'Bi-Weekly':
       return wage * 2 * 12
   elif unit == 'Hour':
       if FT == 'Y':
           return wage * 40 * 4 * 12
       elif FT == 'N':
           return wage * 20 * 4 * 12
Convertwage = np.vectorize(convertwage)

In [17]:
def rangeconvert(w_str):
   w_list = np.array(w_str.split("-"))
   try:
       return (w_list[0].astype(float) + w_list[1].astype(float)) / 2
   except ValueError:
       return w_list[0].astype(float)
Rangeconvert = np.vectorize(rangeconvert)

In [18]:
data2['WAGE_annual'] = Convertwage(Rangeconvert(data2['WAGE_RATE_OF_PAY']), 
                                  data2['WAGE_UNIT_OF_PAY'], data2['FULL_TIME_POSITION'])

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [19]:
data2['PW_WAGE_annual'] = Convertwage(data2['PREVAILING_WAGE'], data2['PW_UNIT_OF_PAY'], data2['FULL_TIME_POSITION'])

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [20]:
data3 = pd.merge(data2, codes, on='NAIC_CODE')

In [21]:
data3.dtypes

CASE_NUMBER                       object
CASE_STATUS                       object
CASE_SUBMITTED            datetime64[ns]
DECISION_DATE             datetime64[ns]
EMPLOYMENT_START_DATE             object
EMPLOYMENT_END_DATE               object
EMPLOYER_NAME                     object
JOB_TITLE                         object
SOC_NAME                          object
NAIC_CODE                        float64
TOTAL WORKERS                    float64
FULL_TIME_POSITION                object
PREVAILING_WAGE                  float64
PW_UNIT_OF_PAY                    object
PW_WAGE_LEVEL                     object
WAGE_RATE_OF_PAY                  object
WAGE_UNIT_OF_PAY                  object
H-1B_DEPENDENT                    object
WILLFUL VIOLATOR                  object
WORKSITE_CITY                     object
WORKSITE_STATE                    object
WORKSITE_POSTAL_CODE              object
decision_date            timedelta64[ns]
WAGE_annual                      float64
PW_WAGE_annual  

In [22]:
def status(S):
   if S == 'CERTIFIED':
       return 1
   else:
       return 0
Status = np.vectorize(status)
data3['Status'] = Status(data3['CASE_STATUS'])

In [23]:
data3.shape

(500433, 27)

In [27]:
def dep(D):
   if D == 'Y':
       return 1
   else:
       return 0
Dep = np.vectorize(dep)
data3['h1b_dependant'] = Dep(data3['H-1B_DEPENDENT'])
data3['violator'] = Dep(data3['WILLFUL VIOLATOR'])
data3['position'] = Dep(data3['FULL_TIME_POSITION'])

In [32]:
data3.dropna(inplace=True)

In [36]:
data3.shape

(479863, 30)

In [34]:
Y = data3.Status
X = data3.loc[:,['TOTAL WORKERS', 'position', 'h1b_dependant',
                'violator', 'WAGE_annual', 'PW_WAGE_annual']]

In [38]:
n = 350000

In [39]:
logit = LogisticRegression(C=1e4)
logit.fit(X.loc[:n], Y.loc[:n])

LogisticRegression(C=10000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [43]:
pred = logit.predict(X.loc[n:])

In [47]:
M = confusion_matrix(pred, Y.loc[n:])

In [46]:
data3.loc[n:].shape

(138959, 30)

In [48]:
M

array([[    35,      2],
       [  3521, 135401]])

In [50]:
print 'Accuracy:', (M[0,0] + M[1,1]) / float(M.sum())*100

Accuracy: 97.4647198094


In [53]:
print 'Coefficients:',logit.coef_
print 'Intercept:', logit.intercept_

Coefficients: [[  2.50811761e-09   1.20419358e-09   9.37227560e-10   2.86335777e-12
    5.70960192e-05  -6.68712834e-08]]
Intercept: [  1.27913426e-09]
